In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [2]:
def generer_admissions(date, service):
    if service == 'Orthopédie':
        base = random.randint(10, 15)
        pic_ete = 20 * np.exp(-((date.timetuple().tm_yday - 180) ** 2) / (2 * 60 ** 2)) #Pic en été
        pic_hiver = 15 * np.exp(-((date.timetuple().tm_yday - 350) ** 2) / (2 * 30 ** 2)) #Pic en hiver
        aleatoire = random.randint(-5, 5)
        return max(0, int(base + pic_ete + pic_hiver + aleatoire))
    elif service == 'Pneumologie':
        base = random.randint(10, 15)
        pic_printemps = 20 * np.exp(-((date.timetuple().tm_yday - 120) ** 2) / (2 * 30 ** 2)) #Pic au printemps
        pic_automne = 20 * np.exp(-((date.timetuple().tm_yday - 300) ** 2) / (2 * 30 ** 2)) #Pic en automne
        pic_hiver = 15 * np.exp(-((date.timetuple().tm_yday - 15) ** 2) / (2 * 30 ** 2)) #Pic en hiver
        aleatoire = random.randint(-5, 5)
        return max(0, int(base + pic_printemps + pic_automne + pic_hiver + aleatoire))

In [3]:
def generer_duree_sejour(service):
    base = 3 if service == 'Orthopédie' else 4
    aleatoire = random.uniform(-1, 1)
    return round(max(1, base + aleatoire), 2)

In [4]:
def generer_lits_disponibles(date, service, admissions, duree_sejour):
    base = 75 if service == 'Orthopédie' else 70
    lits_occupes = admissions * duree_sejour
    lits_disponibles = max(0, int(base - lits_occupes))
    aleatoire = random.randint(-5, 5)  # Ajout d'une variation aléatoire
    return max(0, lits_disponibles + aleatoire)

In [5]:
def generer_personnel(service, type_personnel, date):
    base = 12 if type_personnel == 'medecins' else 25
    conges = 2 * np.sin(2 * np.pi * date.timetuple().tm_yday / 365)
    aleatoire = random.randint(-2, 2)
    return max(0, int(base - conges + aleatoire))

In [6]:
def generer_temperature_paris(date):
    moyenne_mensuelle = [5, 7, 10, 15, 18, 22, 28, 26, 22, 16, 11, 6]
    mois = date.month - 1
    moyenne = moyenne_mensuelle[mois]
    aleatoire = random.uniform(-3, 3)
    return int(moyenne + aleatoire)

In [7]:
def generer_asthme(date, service, admissions):
    if service == 'Orthopédie':
        return 0
    pourcentage_asthme = random.uniform(0.15, 0.20)
    return int(admissions * pourcentage_asthme)

In [8]:
def generer_blessures(date, service):
    if service == 'Pneumologie':
        return 0
    pic = 30 * np.exp(-((date.timetuple().tm_yday - 180) ** 2) / (2 * 60 ** 2)) #Pic en été
    aleatoire = random.randint(0, 10)
    return max(0, int(pic + aleatoire))

In [9]:
def generer_budget(service):
    base = 45000 if service == 'Orthopédie' else 35000
    aleatoire = random.randint(-4000, 4000)
    return base + aleatoire

In [10]:
def generer_stock(base, variation):
    aleatoire = random.randint(-variation, variation)
    return max(0, base + aleatoire)

In [11]:
def generer_consommation_medicaments(admissions, duree_sejour):
    return int(admissions * duree_sejour * random.uniform(0.5, 1.5))
def generer_consommation_consommables(admissions, duree_sejour):
    return int(admissions * duree_sejour * random.uniform(2, 5))

In [12]:
def generer_commandes_medicaments(stock, seuil_alerte):
    if stock < seuil_alerte:
        return random.randint(200, 500)
    else:
        return 0

def generer_commandes_consommables(stock, seuil_alerte):
    if stock < seuil_alerte:
        return random.randint(1000, 2500)
    else:
        return 0

In [13]:
def generer_donnees_hospitalieres(date_debut, date_fin, services):
    dates = pd.date_range(date_debut, date_fin)
    data = []
    for date in dates:
        stocks_medicaments = {service: 1000 for service in services}
        stocks_consommables = {service: 4000 for service in services}
        for service in services:
            admissions = generer_admissions(date, service)
            duree_sejour = generer_duree_sejour(service)
            lits_disponibles = generer_lits_disponibles(date, service, admissions, duree_sejour)
            medecins = generer_personnel(service, 'medecins', date)
            infirmiers = generer_personnel(service, 'infirmiers', date)
            temperature = generer_temperature_paris(date)
            asthme = generer_asthme(date, service, admissions)
            blessures = generer_blessures(date, service)
            budget = generer_budget(service)
            consommation_medicaments = generer_consommation_medicaments(admissions, duree_sejour)
            consommation_consommables = generer_consommation_consommables(admissions, duree_sejour)
            stocks_medicaments[service] = max(0, stocks_medicaments[service] - consommation_medicaments)
            stocks_consommables[service] = max(0, stocks_consommables[service] - consommation_consommables)
            commandes_medicaments = generer_commandes_medicaments(stocks_medicaments[service], 900)
            commandes_consommables = generer_commandes_consommables(stocks_consommables[service], 3800)
            capacite_max_medicaments = 1500 if service == 'Orthopédie' else 1300
            capacite_max_consommables = 6000 if service == 'Orthopédie' else 5000
            data.append([date, service, admissions, duree_sejour, lits_disponibles, medecins, infirmiers, temperature, asthme, blessures, budget, stocks_medicaments[service], stocks_consommables[service], commandes_medicaments, commandes_consommables, capacite_max_medicaments, capacite_max_consommables])
    df = pd.DataFrame(data, columns=['Date', 'Service', 'Admissions', 'Durée séjour (jours)', 'Lits disponibles', 'Médecins', 'Infirmiers', 'Température (°C)', 'Asthme (cas)', 'Blessures (cas)', 'Budget (euros)', 'Stock Médicaments', 'Stock Consommables', 'Commandes Médicaments', 'Commandes Consommables', 'Capacité Max Médicaments', 'Capacité Max Consommables'])
    return df

In [14]:
date_debut = '2022-01-01'
date_fin = '2024-12-31'
services = ['Orthopédie', 'Pneumologie']
df = generer_donnees_hospitalieres(date_debut, date_fin, services)
print(df)

           Date      Service  Admissions  Durée séjour (jours)  \
0    2022-01-01   Orthopédie          11                  3.57   
1    2022-01-01  Pneumologie          30                  3.74   
2    2022-01-02   Orthopédie          16                  3.08   
3    2022-01-02  Pneumologie          26                  4.58   
4    2022-01-03   Orthopédie          19                  3.14   
...         ...          ...         ...                   ...   
2187 2024-12-29  Pneumologie          15                  4.62   
2188 2024-12-30   Orthopédie          20                  3.02   
2189 2024-12-30  Pneumologie           9                  3.21   
2190 2024-12-31   Orthopédie          21                  3.05   
2191 2024-12-31  Pneumologie          17                  4.55   

      Lits disponibles  Médecins  Infirmiers  Température (°C)  Asthme (cas)  \
0                   37        11          22                 2             0   
1                    0        11          22   

In [15]:
df.to_csv('dataSet_Hopital.csv', index=False)